In [22]:
# import libraries
import numpy as np
import pandas as pd
import re
import string
import pickle

In [23]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [24]:
# load stopwords

with open('../static/model/corpora/stopwords/english','r') as file:
    sw = file.read().splitlines()

In [25]:
# load vocab

vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [26]:
# load model

with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [27]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [28]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=['tweet'])
    
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*','',x, flags=re.MULTILINE) for x in x.split()))
    data['tweet'] = data['tweet'].apply(remove_punctuations)
    data["tweet"] = data['tweet'].str.replace('\d+', '', regex=True) # Displayed an error, but worked
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data["tweet"]

<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
C:\Users\manul\AppData\Local\Temp\ipykernel_8404\681685621.py:7: SyntaxWarning: invalid escape sequence '\d'
  data["tweet"] = data['tweet'].str.replace('\d+', '', regex=True) # Displayed an error, but worked


In [29]:
def vectorizer(ds, vocabulary):
    vectorized_list = []

    for sentence in ds:
        sentence_list = np.zeros(len(vocabulary))

        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_list[i] = 1

        vectorized_list.append(sentence_list)

    vectorized_list_new = np.asarray(vectorized_list, dtype=np.float32)
    
    return vectorized_list_new

In [34]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    if prediction == 1:
        return 'Negative'
    else:
        return 'Positive'

In [35]:
txt = "good product I love it"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'Positive'